In [1]:
!pip install psycopg2-binary

In [2]:
import pandas as pd
import sqlalchemy as sql

In [3]:
engine = sql.create_engine('postgresql+psycopg2://postgres:123@db:5432/postgres')

In [4]:
df1 = pd.read_sql(sql='''select "Projection","Description","Projected","Deposit","MonthYear","Payment"*-1 as "Withdrawal",a.amount as "Actual"
from projections_cleaned pc
left join actuals a on datedeposited="MonthYear"
order by projectionid_cleaned asc''', con=engine)

In [5]:
df1

,Projection,Description,Projected,Deposit,MonthYear,Withdrawal,Actual
0,1,Insurance,$214.85,$54.95,2019-04-01,-$1.81,-$2.04
1,1,Insurance,$267.99,$54.95,2019-05-01,-$1.81,-$2.27
2,1,Insurance,$321.13,$54.95,2019-06-01,-$1.81,-$2.18
3,1,Insurance,$374.27,$54.95,2019-07-01,-$1.81,-$2.26
4,1,Insurance,$427.41,$54.95,2019-08-01,-$1.81,-$9.38
5,1,Insurance,$427.41,$54.95,2019-08-01,-$1.81,-$9.38
6,1,Insurance,$427.41,$54.95,2019-08-01,-$1.81,-$9.38
7,1,Insurance,$480.55,$54.95,2019-09-01,-$1.81,None
8,1,Insurance,$533.69,$54.95,2019-10-01,-$1.81,None
9,1,Insurance,$586.83,$54.95,2019-11-01,-$1.81,None


In [6]:
df1.set_index(['Projection','MonthYear','Description']).stack().to_frame()
#close to what the directions were looking for...

0
Projection MonthYear  Description                                       
1          2019-04-01 Insurance                     Projected    $214.85
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$2.04
           2019-05-01 Insurance                     Projected    $267.99
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$2.27
           2019-06-01 Insurance                     Projected    $321.13
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$2.18
           2019-07-01 Insurance                     Projected    $374.27
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$2.26
           2019-08-01 Insurance                     Projected    $427.41
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$9.38
                                                    Projected    $427.41
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$9.38
                                                    Projected    $427.41
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                                                    Actual        -$9.38
           2019-09-01 Insurance                     Projected    $480.55
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
           2019-10-01 Insurance                     Projected    $533.69
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
           2019-11-01 Insurance                     Projected    $586.83
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
           2019-12-01 Insurance                     Projected    $639.97
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
                      Escrowed Tax Payment - County Projected      $2.33
                                                    Withdrawal  -$637.64
           2020-01-01 Insurance                     Projected     $55.47
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
           2020-02-01 Insurance                     Projected    $108.61
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81
2          2020-03-01 Insurance                     Projected    $161.75
                                                    Deposit       $54.95
                                                    Withdrawal    -$1.81

In [11]:
df2 = pd.read_sql(sql='''select *
from (
select distinct "MonthYear","Projection",to_char("MonthYear",'Mon YYYY') as "Month/Year" ,concat('Deposit: ',"Description") as "Activity", "Deposit" as "Estimated", cast(NULL as money) as "Actual", cast(NULL as money) as "Estimated Balance"
from projections_cleaned pc
left join actuals a on datedeposited="MonthYear"
where "Deposit" > cast(0.00 as money)

union all
select "MonthYear","Projection",to_char("MonthYear",'Mon YYYY') as "Month/Year" ,concat('Withdrawal: ',"Description") as "Activity", "Payment"*-1 as "Estimated", a.amount as "Actual", "Projected" as "Estimated Balance"
from projections_cleaned pc
left join actuals a on datedeposited="MonthYear"
) as t
order by "MonthYear"
''', con=engine)

In [12]:
df2.set_index(['Projection','Month/Year','Activity']).drop('MonthYear',axis=1)
#closer to what the directions are looking for!

Estimated  \
Projection Month/Year Activity                                              
1          Apr 2019   Deposit: Insurance                           $54.95   
                      Withdrawal: Insurance                        -$1.81   
           May 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Jun 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Jul 2019   Deposit: Insurance                           $54.95   
                      Withdrawal: Insurance                        -$1.81   
           Aug 2019   Deposit: Insurance                           $54.95   
                      Withdrawal: Insurance                        -$1.81   
                      Withdrawal: Insurance                        -$1.81   
                      Withdrawal: Insurance                        -$1.81   
           Sep 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Oct 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Nov 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Dec 2019   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
                      Withdrawal: Escrowed Tax Payment - County  -$637.64   
           Jan 2020   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
           Feb 2020   Withdrawal: Insurance                        -$1.81   
                      Deposit: Insurance                           $54.95   
2          Mar 2020   Deposit: Insurance                           $54.95   
                      Withdrawal: Insurance                        -$1.81   

                                                                 Actual  \
Projection Month/Year Activity                                            
1          Apr 2019   Deposit: Insurance                           None   
                      Withdrawal: Insurance                      -$2.04   
           May 2019   Withdrawal: Insurance                      -$2.27   
                      Deposit: Insurance                           None   
           Jun 2019   Withdrawal: Insurance                      -$2.18   
                      Deposit: Insurance                           None   
           Jul 2019   Deposit: Insurance                           None   
                      Withdrawal: Insurance                      -$2.26   
           Aug 2019   Deposit: Insurance                           None   
                      Withdrawal: Insurance                      -$9.38   
                      Withdrawal: Insurance                      -$9.38   
                      Withdrawal: Insurance                      -$9.38   
           Sep 2019   Withdrawal: Insurance                        None   
                      Deposit: Insurance                           None   
           Oct 2019   Withdrawal: Insurance                        None   
                      Deposit: Insurance                           None   
           Nov 2019   Withdrawal: Insurance                        None   
                      Deposit: Insurance                           None   
           Dec 2019   Withdrawal: Insurance                        None   
                      Deposit: Insurance                           None   
                      Withdrawal: Escrowed Tax Payment - County    None   
           Jan 2020   Withdrawal: Insurance                        None   
                      Deposit: